In [17]:
import pandas as pd
import numpy as np

from processing import *

In [18]:
path = 'data\dataset_1937770_3.txt'

In [19]:
df = read_and_split_file(path)
df.sample(5)

,text_no_spaces
id,
3,новыйдивандоставканедорого
326,куплюмикроволновкубу
641,"Настанетдень,ияеёброшу"
208,"новыйтостер,черный"
9,куплюноутбукHP


In [20]:
restorer = SpaceRestorer()

test_cases = [
    "куплюайфон14про",
    "ищудомвПодмосковье",
    "сдаюквартирусмебельюитехникой",
    "куплютелевизорPhilips",
    "новыйноутбукAsus",
    "куплюPlaystation5диск",
    "ремонтквартирподключ",
    "новыйдиванIkea",
    "куплюшкаф,рабочий"
]

for case in test_cases:
    print(f"Исходный: {case}")
    print(f"Результат: {restorer.restore_spaces(case)}\n")


Исходный: куплюайфон14про
Результат: куплю айфон 14 про

Исходный: ищудомвПодмосковье
Результат: ищу дом в Подмосковье

Исходный: сдаюквартирусмебельюитехникой
Результат: сдаю квартирусмебельюитехникой

Исходный: куплютелевизорPhilips
Результат: куплю телевизор Philips

Исходный: новыйноутбукAsus
Результат: новый ноутбук Asus

Исходный: куплюPlaystation5диск
Результат: куплю Playstation 5 диск

Исходный: ремонтквартирподключ
Результат: ремонтквартирподключ

Исходный: новыйдиванIkea
Результат: новый диван Ikea

Исходный: куплюшкаф,рабочий
Результат: куплю шкаф, рабочий



In [ ]:
import wikipediaapi
import random
import re
import pandas as pd
from tqdm import tqdm

# Список простых "маскирующихся" юзер-агентов
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/119.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:118.0) Gecko/20100101 Firefox/118.0",
    "SpaceRestorerBot/1.0 (+https://github.com/your-repo)",
]

USER_AGENT = random.choice(USER_AGENTS)

def clean_text(text: str) -> str:
    """Очищает текст от мусора, оставляет буквы/цифры/пробелы"""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^А-Яа-яЁёA-Za-z0-9 ]+', '', text)
    return text.strip()

def get_wiki(lang="ru"):
    """Создаём объект Wikipedia API с user-agent"""
    return wikipediaapi.Wikipedia(
        language=lang,
        user_agent=USER_AGENT
    )

def get_wiki_page(title, lang="ru"):
    """Возвращает текст статьи по названию"""
    wiki = get_wiki(lang)
    page = wiki.page(title)
    if page.exists():
        return clean_text(page.text)
    return ""

def get_random_articles(n=50, lang="ru"):
    """Скачивает случайные статьи"""
    wiki = get_wiki(lang)
    texts = []
    for _ in tqdm(range(n), desc="Случайные статьи"):
        try:
            page = wiki.randompage()
            text = clean_text(page.text)
            if text:
                texts.append(text)
        except Exception:
            continue
    return texts

def make_dataset(text, n_samples=5000, min_words=2, max_words=4):
    """Формируем датасет: без пробелов -> правильный"""
    words = text.split()
    rows = []
    for _ in range(n_samples):
        if len(words) < max_words:
            break
        start = random.randint(0, len(words) - max_words)
        end = start + random.randint(min_words, max_words)
        snippet = words[start:end]
        if len(snippet) < min_words:
            continue
        target_text = " ".join(snippet)
        input_text = "".join(snippet)
        rows.append({"text_no_spaces": input_text, "text_restored": target_text})
    return rows

def generate_synthetic(rows_count=2000):
    """Генерирует синтетические строки: кириллица + латиница + цифры"""
    russian_words = ["Москва", "телефон", "компьютер", "игра", "музыка", "новый", "куплю", "сдам"]
    brands = ["Philips", "Samsung", "iPhone", "Xbox", "PlayStation", "LG", "Asus", "Lenovo", "Dell", "Sony"]
    numbers = ["2023", "14", "13Pro", "One", "360", "5", "X"]

    rows = []
    for _ in range(rows_count):
        rw = random.choice(russian_words)
        br = random.choice(brands)
        num = random.choice(numbers)

        pattern = random.choice([
            f"{rw}{num}",       # Москва2023
            f"{rw}{br}",        # телефонPhilips
            f"{br}{num}",       # iPhone14
            f"{rw}{br}{num}",   # телефонPhilips2023
            f"{br}{num}{rw}"    # iPhone14Москва
        ])

        target = re.sub(r'([А-Яа-яЁё])([0-9])', r'\1 \2', pattern)
        target = re.sub(r'([0-9])([А-Яа-яЁё])', r'\1 \2', target)
        target = re.sub(r'([А-Яа-яЁё])([A-Za-z])', r'\1 \2', target)
        target = re.sub(r'([A-Za-z])([А-Яа-яЁё])', r'\1 \2', target)
        target = re.sub(r'([A-Za-z])([0-9])', r'\1 \2', target)
        target = re.sub(r'([0-9])([A-Za-z])', r'\1 \2', target)

        rows.append({"text_no_spaces": pattern, "text_restored": target})
    return rows

if __name__ == "__main__":
    print(f"Используем User-Agent: {USER_AGENT}\n")

    brand_pages = [
        "Apple",
        "Google",
        "Microsoft",
        "Amazon",
        "Meta",
        "Tesla",
        "Samsung",
        "Huawei",
        "Xiaomi",
        "Intel",
        "AMD",
        "PlayStation",
        "Xbox",
        "Nintendo",
        "LG",
        "Philips",
        "Sony",
        "Dell",
        "HP",
        "Asus",
        "Lenovo",
    ]

    print("Скачиваем брендовые статьи...")
    brand_texts = [get_wiki_page(title, lang="ru") for title in brand_pages]

    print("Скачиваем случайные статьи...")
    random_texts = get_random_articles(n=100, lang="ru")

    print("Формируем общий корпус...")
    full_text = " ".join(brand_texts + random_texts)

    print("Создаём датасет из Википедии...")
    wiki_rows = make_dataset(full_text, n_samples=8000)

    print("Создаём синтетический датасет...")
    synthetic_rows = generate_synthetic(rows_count=2000)

    # объединяем
    all_rows = wiki_rows + synthetic_rows
    df = pd.DataFrame(all_rows)

    df.to_csv("wiki_dataset.csv", index=False, encoding="utf-8", sep=';')
    print("Готово! Сохранено в wiki_dataset.csv")


Используем User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:118.0) Gecko/20100101 Firefox/118.0

Скачиваем брендовые статьи...
Скачиваем случайные статьи...


Случайные статьи: 100%|██████████| 100/100 [00:00<?, ?it/s]

Формируем общий корпус...
Создаём датасет из Википедии...
Создаём синтетический датасет...
Готово! Сохранено в wiki_dataset.csv
